# Working with sentinel-2 data

## AIM

The aim is to batch clip processed sentinel imagery with the extend of the study area

## DATA MANAGMENT

In [1]:
import os
from glob import glob

# import gui tkinter
from tkinter import Tk
import tkinter.filedialog as tkfd

### Set input and output directories

A **tkinter GUI** is loaded:
- to locate and select the **input directory**,
- to define the **output directory**, and
- to locate a GeoJSON file, defining the **bouding box of the study area**.

The tkinter input is printed to the console.

In [2]:
# tkinter GUI
root = Tk()
input_dir = tkfd.askdirectory(title = "Select input folder", initialdir = "C:/")
output_dir = tkfd.askdirectory(title = "Select output folder", initialdir = "C:/")
study_area = tkfd.askopenfilename(title = "Select extend study area", 
                                  initialdir = "C:/", 
                                  filetypes = (("GeoJSON", "*.geojson"), ("all files", "*.*")))
root.destroy()

# print statements: tkinter input is printed to the console
print("The selected input folder is:")
print("-", input_dir)
print(" ")
print("The selected output folder is:")
print("-", output_dir)
print(" ")
print("The selected study area bounding box is located at:")
print("-", study_area)

The selected input folder is:
- C:/SS
 
The selected output folder is:
- C:/SSClip
 
The selected study area bounding box is located at:
- C:/WERKMAP/Anaconda/sentinelsat/geojson/Sentinelsat.geojson


## CLIPPING with STUDY AREA

In [3]:
import osgeo
from osgeo import gdal
from rasterstats import zonal_stats

In [4]:
# Enable GDAL/OGR exceptions and set GDAL data
gdal.UseExceptions()
osgeo.gdal.SetConfigOption("GDAL_DATA", "C:/Users/jeroen.dereu@inbo.be/Software/gdal-2.1.3/data")

### Check for NO DATA

In [5]:
folders_to_check = []
folders_to_check_count = 0
for filename in os.listdir(input_dir):
    folder = input_dir + "/" + filename + "/"
    folders_to_check.append(folder)
    folders_to_check_count = folders_to_check_count + 1

print("Checking", folders_to_check_count, "datasets for data/no data within the study area bounding box...")

folders_to_process = []
folders_to_process_count = 0
no_action_folders = []
no_action_folders_count = 0
for folder in folders_to_check:
    for path in glob(os.path.join(folder, "**", "*overview.tif"), recursive = True):
        mean = zonal_stats(study_area, path, stats = "mean", nodata = 0)
        if mean != [{'mean': None}]:
            print("+", folder, ": Data within study area, folder is kept for further processing!")
            folders_to_process.append(folder)
            folders_to_process_count = folders_to_process_count + 1
        else:
            print("-", folder, ": No data within the study area, folder is dropped from the selection.")
            no_action_folders.append(folder)
            no_action_folders_count = no_action_folders_count + 1
            
print(" ")
print("SUMMARY")
print("---")
print(no_action_folders_count, "datasets contain no data and are dropped from the selection:")
for no_action_folder in no_action_folders:
    print("-", no_action_folder)
print(" ")
print("Processing will continue with", folders_to_process_count, "datasets:")
for folder_to_process in folders_to_process:
    print("+", folder_to_process)

Checking 8 datasets for data/no data within the study area bounding box...
- C:/SS/20160703/ : No data within the study area, folder is dropped from the selection.
- C:/SS/20160707/ : No data within the study area, folder is dropped from the selection.
+ C:/SS/20160720/ : Data within study area, folder is kept for further processing!
- C:/SS/20160723/ : No data within the study area, folder is dropped from the selection.
+ C:/SS/20160809/ : Data within study area, folder is kept for further processing!
+ C:/SS/20160812/ : Data within study area, folder is kept for further processing!
+ C:/SS/20160822/ : Data within study area, folder is kept for further processing!
- C:/SS/20160826/ : No data within the study area, folder is dropped from the selection.
 
SUMMARY
---
4 datasets contain no data and are dropped from the selection:
- C:/SS/20160703/
- C:/SS/20160707/
- C:/SS/20160723/
- C:/SS/20160826/
 
Processing will continue with 4 datasets:
+ C:/SS/20160720/
+ C:/SS/20160809/
+ C:/SS/

### Clip bands

In [6]:
for folder in folders_to_process:
    print("Start clipping bands in", folder, "...")
    for path in glob(os.path.join(folder, "merged_bands", "*.tif"), recursive = True):
        basename = os.path.basename(path)
        filename, file_extention = os.path.splitext(basename)
        date = filename[0:7]
        if not os.path.isdir(output_dir + "/" + date):
            os.mkdir(output_dir + "/" + date)
        if not os.path.isdir(output_dir + "/" + date + "/merged_bands"):
            os.mkdir(output_dir + "/" + date + "/merged_bands")    
        output_folder = output_dir + "/" + date + "/merged_bands" + "/"
        output_file = output_folder + filename + ".tif"
        gdal.Warp(destNameOrDestDS = output_file, 
                  srcDSOrSrcDSTab = path, 
                  cropToCutline = True, 
                  cutlineDSName = study_area)
    print("Finished clipping bands in", folder, "!")
print("All datastes processed. Finished clipping bands!")

Start clipping bands in C:/SS/20160720/ ...
Finished clipping bands in C:/SS/20160720/ !
Start clipping bands in C:/SS/20160809/ ...
Finished clipping bands in C:/SS/20160809/ !
Start clipping bands in C:/SS/20160812/ ...
Finished clipping bands in C:/SS/20160812/ !
Start clipping bands in C:/SS/20160822/ ...
Finished clipping bands in C:/SS/20160822/ !
All datastes processed. Finished clipping bands!


### Clip RGB

In [7]:
for folder in folders_to_process:
    print("Start clipping RGB-image in", folder, "...")
    for path in glob(os.path.join(folder, "RGB", "*.tif"), recursive = True):
        basename = os.path.basename(path)
        filename, file_extention = os.path.splitext(basename)
        date = filename[0:7]
        if not os.path.isdir(output_dir + "/" + date):
            os.mkdir(output_dir + "/" + date)
        if not os.path.isdir(output_dir + "/" + date + "/RGB"):
            os.mkdir(output_dir + "/" + date + "/RGB")    
        output_folder = output_dir + "/" + date + "/RGB" + "/"
        output_file = output_folder + filename + ".tif"
        gdal.Warp(destNameOrDestDS = output_file, 
                  srcDSOrSrcDSTab = path, 
                  cropToCutline = True, 
                  cutlineDSName = study_area)
    print("Finished clipping RGB-image in", folder, "!")
print("All datastes processed. Finished clipping RGB-imagery!")

Start clipping RGB-image in C:/SS/20160720/ ...
Finished clipping RGB-image in C:/SS/20160720/ !
Start clipping RGB-image in C:/SS/20160809/ ...
Finished clipping RGB-image in C:/SS/20160809/ !
Start clipping RGB-image in C:/SS/20160812/ ...
Finished clipping RGB-image in C:/SS/20160812/ !
Start clipping RGB-image in C:/SS/20160822/ ...
Finished clipping RGB-image in C:/SS/20160822/ !
All datastes processed. Finished clipping RGB-imagery!


### Clip NDVI

In [8]:
for folder in folders_to_process:
    print("Start clipping NDVI-image in", folder, "...")
    for path in glob(os.path.join(folder, "NDVI", "*.tif"), recursive = True):
        basename = os.path.basename(path)
        filename, file_extention = os.path.splitext(basename)
        date = filename[0:7]
        if not os.path.isdir(output_dir + "/" + date):
            os.mkdir(output_dir + "/" + date)
        if not os.path.isdir(output_dir + "/" + date + "/NDVI"):
            os.mkdir(output_dir + "/" + date + "/NDVI")    
        output_folder = output_dir + "/" + date + "/NDVI" + "/"
        output_file = output_folder + filename + ".tif"
        gdal.Warp(destNameOrDestDS = output_file, 
                  srcDSOrSrcDSTab = path, 
                  cropToCutline = True, 
                  cutlineDSName = study_area)
    print("Finished clipping NDVI-image in", folder, "!")
print("All datastes processed. Finished clipping NDVI-imagery!")

Start clipping NDVI-image in C:/SS/20160720/ ...
Finished clipping NDVI-image in C:/SS/20160720/ !
Start clipping NDVI-image in C:/SS/20160809/ ...
Finished clipping NDVI-image in C:/SS/20160809/ !
Start clipping NDVI-image in C:/SS/20160812/ ...
Finished clipping NDVI-image in C:/SS/20160812/ !
Start clipping NDVI-image in C:/SS/20160822/ ...
Finished clipping NDVI-image in C:/SS/20160822/ !
All datastes processed. Finished clipping NDVI-imagery!


In [9]:
print("All clipping done, process finished!")

All clipping done, process finished!
